In [2]:
from copy import deepcopy
from abc import ABCMeta, abstractmethod

In [3]:
class Event:
    pass


class TickEvent(Event):
    def __init__(self, instrument, time, bid, ask):
        self.type = "TICK"
        self.instrument = instrument
        self.time = time
        self.bid = bid
        self.ask = ask
        
    def __str__(self):
        return "Type: %s, Instrument: %s, Time: %s, Bid: %s, Ask: %s" %(str(self.type),
                                                                        str(self.instrument),
                                                                        str(self.time),
                                                                        str(self.bid),
                                                                        str(self.ask))
    
    def __repr__(self):
        return str(self)
    

class SignalEvent(Event):
    def __init__(self, instrument, orderType, orderSide, time):
        self.type = "SIGNAL"
        self.instrument = instrument
        self.orderType = orderType
        self.orderSide = orderSide
        self.time = time
        
    def __str__(self):
        return "Type: %s, Instrument: %s, Order Type: %s, Side: %s" % (str(self.type),
                                                                       str(self.instrument),
                                                                       str(self.OrderType),
                                                                       str(self.OrderSide))
    
    def __repr__(self):
        str(self)

class OrderEvent(Event):
    def __init__(self, instrument, units, orderType, orderSide):
        self.type = "ORDER"
        self.instrument = instrument
        self.units = units
        self.orderType = orderType
        self.orderSide = orderSide
        
    def __str__(self):
        return "Type: %s, Instrument: %s, Units: %s, Order Type: %s, Side: %s" %(
            str(self.type),
                                                                                str(self.instrument),
                                                                                str(self.units),
                                                                                str(self.orderType),
                                                                                str(self.orderSide)
                                                                                )

In [ ]:
class MovingAverageCrossStrategy:
    
    def __init__(self, pairs, events, shortWindow=50, longWindow=200):
        self.pairs = pairs
        self.events = events
        self.shortWindow = shortWindow
        self.longWindow = longWindow
        self.pairsDict = self.CreatePairsDict()
    
    def CreatePairsDict(self):
        attrDict = {"ticks": 0, 
                    "invested": False,
                    "shortSMA": None,
                    "longSMA": None
                    }
        pairsDict = {}
        for p in self.pairs:
            pairsDict[p] = copy.deepcopy(attrDict)
        return pairsDict
    
    
    def CalculateSignals(self, event):
        if event.type == 'TICK':
            pair = event.instrument
            price = event.bid
            pd = self.pairsDict[pair]
            if pd["ticks"] == 0:
                pd["shortSMA"] = price
                pd["longSMA"] = price
            else:
                pd["shortSMA"] = self.CalcRollingSMA(pd["shortSMA"],
                                                     self.shortWindow,
                                                     price)
                pd["longSMA"] = self.CalcRollingSMA(pd["longSMA"],
                                                    self.longWindow,
                                                    price)
            if pd["ticks"] > self.shortWindow:
                if pd["shortSMA"] > pd["longSMA"] and not pd["invested"]:
                    signal = SignalEvent(pair, "market", "buy", event.time)
                    self.events.put(signal)